# Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import statsmodels.api as sm

from IPython.display import display, Markdown
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression

from statsmodels.tools.eval_measures import rmse # Para calcular el error

from math import sqrt
import scipy.cluster.hierarchy as hac

%matplotlib inline
%config IPCompleter.greedy=True
warnings.filterwarnings('ignore')

# CLASIFICADORES

# Carga de datos para clasificacion


In [145]:
mejora = 30
edad_min=20
edad_max=40

#cargamos fichero
datos = pd.read_csv('rawTodos.csv', na_values='na')
print("lineas Totales-->"+str(len(datos)))
datos=datos.drop(['bef_percent_saves','bef_secondary_position'], axis=1)
#Filtrado por minutos
datos=datos[datos.bef_mins_played>=400]
#Fitramos Score no supere 10


#Filtrado edades
datos=datos[datos.bef_age <= edad_max]
datos=datos[datos.bef_age >= edad_min]

#Si queremos filtrar por posicion TODAS LAS POSICIONES --> 'GK','DC','DL','DR','DMC','DML','DMR','MC','ML','MR','AMC','AMR','AML','FW','FWL','FWR'
array = ['DC']
#array = ['GK']

datos=datos.loc[datos.bef_position.isin(array)] 

datos=datos[datos.bef_Score <= 10]
datos=datos[datos.aft_Score <= 10]
datos=datos[datos.bef_Score >= 1]
datos=datos[datos.aft_Score >= 1]
datos= datos.rename(columns={'inc_score': 'label'})
cont=0
for a in datos['label']:
    if(a>30): cont+=1
print(f'{cont}')


"""
#PARA PORTEROS
datos=datos[['label', 'bef_age','bef_TIER', 'bef_Score','xg_faced', 'xg_faced_on_target', 'xg_saved','bef_passes_total_own_half_accurate',
             'bef_passes_total_own_half', 'bef_passes_total_opp_half_accurate', 'bef_passes_total_opp_half',
             'bef_passes_total_final_third_accurate', 'bef_passes_total_final_third','bef_saves_in_box',
             'bef_saves_out_of_box', 'bef_goals_received', 'bef_goals_received_in_box', 'bef_goals_received_out_box',
             'bef_non_penalty_goals_received', 'bef_saved_shots_faced_total', 'bef_saved_shots_faced_in_box',
             'bef_saved_shots_faced_out_box', 'bef_shots_received_on_target_total', 
             'bef_shots_received_on_target_in_box', 'bef_shots_received_on_target_out_box',
             'bef_non_penalty_shots_received_on_target', 'bef_missed_shots_faced_total', 'bef_missed_shots_faced_in_box', 
             'bef_missed_shots_faced_out_box', 'bef_shots_on_post_faced_total', 'bef_shots_on_post_faced_in_box',
             'bef_shots_on_post_faced_out_box', 'bef_shots_received_off_target_total', 'bef_shots_received_off_target_in_box',
             'bef_shots_received_off_target_out_box', 'bef_shots_received_total', 'bef_shots_received_in_box',
             'bef_shots_received_out_box', 'bef_non_penalty_shots_received','bef_yellow_cards','bef_second_yellow_cards',
                                       'bef_red_cards']]
"""
#filtramos el incremento en mejora o no mejora
datos.loc[datos.label <mejora, 'label'] = 0
datos.loc[datos.label >=mejora, 'label'] = 1




# entrenamiento y test
train_features, test_features = train_test_split(datos, test_size=0.35)

train_labels = train_features['label']
test_labels = test_features['label']
#BALANCEO

idxs_pos = pd.DataFrame(train_labels)[train_labels==1].index
idxs_neg = pd.DataFrame(pd.DataFrame(train_labels)[train_labels==0]).sample(n=len(idxs_pos), replace=False, random_state=0).index
idxs_balanced = np.concatenate((idxs_pos,idxs_neg)) #numero de filas excogidas(la 4,19, ....)
train_labels= pd.DataFrame(train_labels).loc[idxs_balanced].values
train_features= pd.DataFrame(train_features).loc[idxs_balanced]
print("Train Mejora >=",mejora," : ",str(len(idxs_pos)))
print("Train Mejora < ",mejora," : ",str(len(idxs_neg)))


a = pd.DataFrame(test_labels)[test_labels==1].index.values
b = pd.DataFrame(pd.DataFrame(test_labels)[test_labels==0]).sample(n=len(a), replace=False, random_state=0).index
c = np.concatenate((a,b)) #numero de filas excogidas(la 4,19, ....)
test_labels= pd.DataFrame(test_labels).loc[c].values
test_features= pd.DataFrame(test_features).loc[c]
print("Test Mejora >=",mejora," : ",str(len(a)))
print("Test Mejora < ",mejora," : ",str(len(b)))

test_features_copia=test_features
train_features_copia=train_features
#BORRADO DE COLUMMNAS IMNECESARIAS
#PARA JUGADORES DE CAMPO
train_features = train_features.drop([#'aft_Score2',
                    'aft_TIER',
                    'mins_played','bef_mins_played',
                                      'bef_player_id','bef_team_id','bef_season_id',
                                      'bef_first_name','bef_Team','bef_year',
                                      'bef_name','bef_league_id',
                                      'bef_passes_accurate_in_team','bef_source','bef_touches_in_team',
                                      'bef_loaded_at','bef_percent_passes_own_half','bef_percent_passes_opp_half',
                                      'bef_percent_passes_final_third',
                                      'bef_Score2','bef_lined_up', 'bef_subbed_in', 'bef_subbed_out',
                                      'bef_games_played','aft_Score',
                                      'bef_position','bef_league_id','bef_possession', 'bef_calculated_possession',
                                        'xg_faced', 'xg_faced_on_target', 'xg_saved',
                                      'bef_passes_total_own_half_accurate', 'bef_passes_total_own_half', 
                                      'bef_passes_total_opp_half_accurate', 'bef_passes_total_opp_half', 
                                      'bef_passes_total_final_third_accurate', 'bef_passes_total_final_third',
                                      'bef_saves_in_box', 'bef_saves_out_of_box', 'bef_goals_received', 
                                      'bef_goals_received_in_box', 'bef_goals_received_out_box', 
                                      'bef_non_penalty_goals_received', 'bef_saved_shots_faced_total',
                                      'bef_saved_shots_faced_in_box', 'bef_saved_shots_faced_out_box',
                                      'bef_shots_received_on_target_total', 'bef_shots_received_on_target_in_box',
                                      'bef_shots_received_on_target_out_box', 'bef_non_penalty_shots_received_on_target',
                                      'bef_missed_shots_faced_total', 'bef_missed_shots_faced_in_box',
                                      'bef_missed_shots_faced_out_box', 'bef_shots_on_post_faced_total', 
                                      'bef_shots_on_post_faced_in_box', 'bef_shots_on_post_faced_out_box',
                                      'bef_shots_received_off_target_total', 'bef_shots_received_off_target_in_box',
                                      'bef_shots_received_off_target_out_box', 'bef_shots_received_total',
                                      'bef_shots_received_in_box', 'bef_shots_received_out_box',
                                      'bef_non_penalty_shots_received','bef_events_zone_p1','bef_events_zone_p2',
                                      'bef_events_zone_p3', 'bef_events_zone_p4','bef_events_zone_p5', 'bef_events_zone_p6',
                                      'bef_events_zone_p7', 'bef_events_zone_p8',
                                      'bef_events_zone_p9', 'bef_events_30_zone_p1',
                                       'bef_events_30_zone_p2', 'bef_events_30_zone_p3',
                                      'bef_events_30_zone_p4', 'bef_events_30_zone_p5',
                                     'bef_events_30_zone_p6', 'bef_events_30_zone_p7',
                                     'bef_events_30_zone_p8', 'bef_events_30_zone_p9',
                                     'bef_events_30_zone_p10', 'bef_events_30_zone_p11',
                                     'bef_events_30_zone_p12', 'bef_events_30_zone_p13',
                                     'bef_events_30_zone_p14', 'bef_events_30_zone_p15',
                                     'bef_events_30_zone_p16', 'bef_events_30_zone_p17',
                                     'bef_events_30_zone_p18', 'bef_events_30_zone_p19',
                                     'bef_events_30_zone_p20', 'bef_events_30_zone_p21',
                                     'bef_events_30_zone_p22', 'bef_events_30_zone_p23',
                                     'bef_events_30_zone_p24', 'bef_events_30_zone_p25',
                                     'bef_events_30_zone_p26', 'bef_events_30_zone_p27',
                                     'bef_events_30_zone_p28', 'bef_events_30_zone_p29',
                                     'bef_events_30_zone_p30','bef_yellow_cards','bef_second_yellow_cards',
                                       'bef_red_cards'], axis=1)
test_features = test_features.drop([#'aft_Score2',
                    'aft_TIER',
                    'mins_played','bef_mins_played',
                                      'bef_player_id','bef_team_id','bef_season_id',
                                      'bef_first_name','bef_Team','bef_year',
                                      'bef_name','bef_league_id',
                                      'bef_passes_accurate_in_team','bef_source','bef_touches_in_team',
                                      'bef_loaded_at','bef_percent_passes_own_half','bef_percent_passes_opp_half',
                                      'bef_percent_passes_final_third',
                                      'bef_Score2','bef_lined_up', 'bef_subbed_in', 'bef_subbed_out',
                                      'bef_games_played','aft_Score',
                                      'bef_position','bef_league_id','bef_possession', 'bef_calculated_possession',
                                        'xg_faced', 'xg_faced_on_target', 'xg_saved',
                                      'bef_passes_total_own_half_accurate', 'bef_passes_total_own_half', 
                                      'bef_passes_total_opp_half_accurate', 'bef_passes_total_opp_half', 
                                      'bef_passes_total_final_third_accurate', 'bef_passes_total_final_third',
                                      'bef_saves_in_box', 'bef_saves_out_of_box', 'bef_goals_received', 
                                      'bef_goals_received_in_box', 'bef_goals_received_out_box', 
                                      'bef_non_penalty_goals_received', 'bef_saved_shots_faced_total',
                                      'bef_saved_shots_faced_in_box', 'bef_saved_shots_faced_out_box',
                                      'bef_shots_received_on_target_total', 'bef_shots_received_on_target_in_box',
                                      'bef_shots_received_on_target_out_box', 'bef_non_penalty_shots_received_on_target',
                                      'bef_missed_shots_faced_total', 'bef_missed_shots_faced_in_box',
                                      'bef_missed_shots_faced_out_box', 'bef_shots_on_post_faced_total', 
                                      'bef_shots_on_post_faced_in_box', 'bef_shots_on_post_faced_out_box',
                                      'bef_shots_received_off_target_total', 'bef_shots_received_off_target_in_box',
                                      'bef_shots_received_off_target_out_box', 'bef_shots_received_total',
                                      'bef_shots_received_in_box', 'bef_shots_received_out_box',
                                      'bef_non_penalty_shots_received','bef_events_zone_p1','bef_events_zone_p2',
                                      'bef_events_zone_p3', 'bef_events_zone_p4','bef_events_zone_p5', 'bef_events_zone_p6',
                                      'bef_events_zone_p7', 'bef_events_zone_p8',
                                      'bef_events_zone_p9', 'bef_events_30_zone_p1',
                                       'bef_events_30_zone_p2', 'bef_events_30_zone_p3',
                                      'bef_events_30_zone_p4', 'bef_events_30_zone_p5',
                                     'bef_events_30_zone_p6', 'bef_events_30_zone_p7',
                                     'bef_events_30_zone_p8', 'bef_events_30_zone_p9',
                                     'bef_events_30_zone_p10', 'bef_events_30_zone_p11',
                                     'bef_events_30_zone_p12', 'bef_events_30_zone_p13',
                                     'bef_events_30_zone_p14', 'bef_events_30_zone_p15',
                                     'bef_events_30_zone_p16', 'bef_events_30_zone_p17',
                                     'bef_events_30_zone_p18', 'bef_events_30_zone_p19',
                                     'bef_events_30_zone_p20', 'bef_events_30_zone_p21',
                                     'bef_events_30_zone_p22', 'bef_events_30_zone_p23',
                                     'bef_events_30_zone_p24', 'bef_events_30_zone_p25',
                                     'bef_events_30_zone_p26', 'bef_events_30_zone_p27',
                                     'bef_events_30_zone_p28', 'bef_events_30_zone_p29',
                                     'bef_events_30_zone_p30','bef_yellow_cards','bef_second_yellow_cards',
                                       'bef_red_cards'], axis=1)

#train_features = train_features.drop(['label','aft_Score','bef_mins_played'], axis=1).values
#test_features = test_features.drop(['label','aft_Score','bef_mins_played'], axis=1).values

train_features =train_features.drop('label', axis=1)
test_features =test_features.drop('label', axis=1)
columnas = list(train_features)

#datos['label']
print("Entrenamiento-->"+str(len(train_features)))
print("Test"+str(len(test_features)))
print(str(len(columnas)))

columnas



lineas Totales-->58165
1059
Train Mejora >= 30  :  672
Train Mejora <  30  :  672
Test Mejora >= 30  :  387
Test Mejora <  30  :  387
Entrenamiento-->1344
Test774
76


['bef_passes_accurate',
 'bef_passes_total',
 'bef_percent_passes',
 'bef_passes_into_opponent_box',
 'bef_passes_key',
 'bef_open_play_key_passes',
 'bef_open_play_passes_into_opponent_box',
 'bef_long_passes_successful',
 'bef_long_passes_total',
 'bef_percent_long_passes',
 'bef_crosses_successful',
 'bef_crosses_total',
 'bef_percent_crosses',
 'bef_tackles_successful_with_possession',
 'bef_tackles_successful_without_possession',
 'bef_tackles_unsuccessful',
 'bef_tackles_total',
 'bef_adjusted_tackles',
 'bef_tackles_was_dribbled',
 'bef_interceptions',
 'bef_adjusted_interceptions',
 'bef_clearance_total',
 'bef_adjusted_clearances',
 'bef_fouls_committed',
 'bef_aerials_won',
 'bef_aerials_total',
 'bef_percent_aerials',
 'bef_assists',
 'bef_dribbles_won',
 'bef_dispossessed',
 'bef_recoveries',
 'bef_adjusted_recoveries',
 'bef_chances_created',
 'bef_through_balls',
 'bef_through_balls_accurate',
 'bef_scoring_contribution',
 'bef_touches',
 'bef_touches_in_opponent_box',
 '

### Gradient Boosting

In [97]:
#Gradient Boosting

gbc = GradientBoostingClassifier(loss='deviance', random_state=0)
gbc.fit(train_features, train_labels)
prediccion = gbc.predict(test_features)
report = classification_report(test_labels, prediccion)
rms = sqrt(mean_squared_error(test_labels, prediccion))
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.7264631043256997
              precision    recall  f1-score   support

         0.0       0.75      0.68      0.71       393
         1.0       0.71      0.77      0.74       393

   micro avg       0.73      0.73      0.73       786
   macro avg       0.73      0.73      0.73       786
weighted avg       0.73      0.73      0.73       786



,tn,fp,fn,tp
0,267,126,89,304


### Gradient Boosting con validacion cruzada

In [186]:
#Gradient Boosting con validacion cruzada
params = [{'loss': ['deviance', 'exponential']}]
gbcc = GridSearchCV(gbc, params, cv=5, scoring='recall', verbose=10, n_jobs=3)
gbcc.fit(train_features, train_labels)
gbc = gbcc
prediccion = gbc.predict(test_features)
report = classification_report(test_labels, prediccion)
rms = sqrt(mean_squared_error(test_labels, prediccion))
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    4.5s
[Parallel(n_jobs=3)]: Done   7 out of  10 | elapsed:    5.8s remaining:    2.4s
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    6.4s finished


Correct classification rate: 1.0
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



ValueError: cannot reshape array of size 1 into shape (1,4)


### Logistic Regression

In [171]:
import sys
np.set_printoptions(threshold=sys.maxsize)

modelo = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.5, fit_intercept=True, intercept_scaling=1, class_weight='balanced', random_state=None, 
                            solver='lbfgs', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=3)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
proba =  modelo.predict_proba(test_features)
#Jugadores de interes
idx = pd.DataFrame(proba)[proba[:,1]>0.9].index
#for a in idx:
   # print(test_features_copia[test_features_copia.index==a])
cota  = 0.56 # si el 1 está por debajo de esto, considerarlo un cero

prediccion = np.round(prediccion)
prediccion= pd.DataFrame(prediccion)

cambiame = pd.DataFrame(proba)[proba[:,1]<cota].index
for i in cambiame:
    prediccion[prediccion.index==i]=0
    
report = classification_report(test_labels, prediccion.values)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
cm


Correct classification rate: 0.7622739018087855
              precision    recall  f1-score   support

         0.0       0.75      0.79      0.77       387
         1.0       0.78      0.74      0.76       387

   micro avg       0.76      0.76      0.76       774
   macro avg       0.76      0.76      0.76       774
weighted avg       0.76      0.76      0.76       774



,tn,fp,fn,tp
0,304,83,101,286


### Random Forest Regressor

In [172]:
#Random Forest Regressor
modelo = RandomForestRegressor(n_estimators=350, oob_score = True, random_state=0, n_jobs=3)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
prediccion = np.round(prediccion)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)


Correct classification rate: 0.7364341085271318
              precision    recall  f1-score   support

         0.0       0.75      0.70      0.73       387
         1.0       0.72      0.77      0.75       387

   micro avg       0.74      0.74      0.74       774
   macro avg       0.74      0.74      0.74       774
weighted avg       0.74      0.74      0.74       774



,tn,fp,fn,tp
0,271,116,88,299


### RamdomForest clasificador

In [100]:
#RamdomForest clasificador
modelo = RandomForestClassifier(criterion='gini', random_state=0)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.6908396946564885
              precision    recall  f1-score   support

         0.0       0.68      0.74      0.70       393
         1.0       0.71      0.65      0.68       393

   micro avg       0.69      0.69      0.69       786
   macro avg       0.69      0.69      0.69       786
weighted avg       0.69      0.69      0.69       786



,tn,fp,fn,tp
0,289,104,139,254


### GridSearchCV

In [173]:
params = [{'kernel': ['rbf'], 'gamma': [0.01], 'C': [0.001, 0.01, 0.1, 1, 10]}, 
              {'kernel': ['linear'], 'gamma': [0.01],  'C':  [0.001, 0.01, 0.1, 1, 10]}
             ]
#Executa grid search com cross validation
modelo = GridSearchCV(svm.SVC(C=1), params, cv=5, scoring='recall', verbose=10, n_jobs=3)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    5.9s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:    6.8s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    7.3s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:    8.4s
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:    9.0s
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    9.8s
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   10.5s
[Parallel(n_jobs=3)]: Done  50 out of  50 | elapsed:   11.5s finished


Correct classification rate: 0.7493540051679587
              precision    recall  f1-score   support

         0.0       0.84      0.62      0.71       387
         1.0       0.70      0.88      0.78       387

   micro avg       0.75      0.75      0.75       774
   macro avg       0.77      0.75      0.74       774
weighted avg       0.77      0.75      0.74       774



,tn,fp,fn,tp
0,239,148,46,341


### SI SE QUIERE BALANCEO EJECUTAR SIGUIENTE LINEA

In [39]:

idxs_pos = pd.DataFrame(train_labels)[train_labels==1].index
idxs_neg = pd.DataFrame(pd.DataFrame(train_labels)[train_labels==0]).sample(n=len(idxs_pos), replace=False, random_state=0).index
idxs_balanced = np.concatenate((idxs_pos,idxs_neg)) #numero de filas excogidas(la 4,19, ....)
train_labels= pd.DataFrame(train_labels).loc[idxs_balanced].values
train_features= pd.DataFrame(train_features).loc[idxs_balanced]

print(str(len(idxs_pos)))
print(str(len(idxs_neg)))

a = pd.DataFrame(test_labels)[test_labels==1].index.values
b = pd.DataFrame(pd.DataFrame(test_labels)[test_labels==0]).sample(n=len(a), replace=False, random_state=0).index
c = np.concatenate((a,b)) #numero de filas excogidas(la 4,19, ....)
test_labels= pd.DataFrame(test_labels).loc[c].values
test_features= pd.DataFrame(test_features).loc[c]

print(str(len(a)))
print(str(len(b)))


698
698
361
361


### GaussianProcessClassifier

In [174]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0 * RBF(1.0)
modelo = GaussianProcessClassifier(kernel=kernel,random_state=0)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.7635658914728682
              precision    recall  f1-score   support

         0.0       0.79      0.72      0.75       387
         1.0       0.74      0.81      0.77       387

   micro avg       0.76      0.76      0.76       774
   macro avg       0.77      0.76      0.76       774
weighted avg       0.77      0.76      0.76       774



,tn,fp,fn,tp
0,279,108,75,312


#### Gaussian Naive Bayes

In [101]:
from sklearn.naive_bayes import GaussianNB
modelo= GaussianNB()
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.6653944020356234
              precision    recall  f1-score   support

         0.0       0.73      0.53      0.61       393
         1.0       0.63      0.80      0.70       393

   micro avg       0.67      0.67      0.67       786
   macro avg       0.68      0.67      0.66       786
weighted avg       0.68      0.67      0.66       786



,tn,fp,fn,tp
0,209,184,79,314


### Bernoulli Naive Bayes

In [102]:
from sklearn.naive_bayes import BernoulliNB
modelo = BernoulliNB()
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)


Correct classification rate: 0.5572519083969466
              precision    recall  f1-score   support

         0.0       0.55      0.66      0.60       393
         1.0       0.57      0.45      0.50       393

   micro avg       0.56      0.56      0.56       786
   macro avg       0.56      0.56      0.55       786
weighted avg       0.56      0.56      0.55       786



,tn,fp,fn,tp
0,261,132,216,177


### DecisionTreeClassifier

In [175]:
from sklearn import tree
modelo = tree.DecisionTreeClassifier(random_state=0)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import collections
dot_data = tree.export_graphviz(modelo,
                                feature_names=columnas,
                                out_file=None,
                                filled=True,
                                rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)

colors = ('turquoise', 'orange')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])

graph.write_png('tree.png')

Correct classification rate: 0.6498708010335917
              precision    recall  f1-score   support

         0.0       0.65      0.64      0.65       387
         1.0       0.65      0.66      0.65       387

   micro avg       0.65      0.65      0.65       774
   macro avg       0.65      0.65      0.65       774
weighted avg       0.65      0.65      0.65       774



,tn,fp,fn,tp
0,248,139,132,255


ModuleNotFoundError: No module named 'pydotplus'

### AdaBoostClassifier

In [104]:
from sklearn.ensemble import AdaBoostClassifier
modelo = AdaBoostClassifier(tree.DecisionTreeClassifier(),
                         algorithm="SAMME",
                         n_estimators=500)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.6628498727735369
              precision    recall  f1-score   support

         0.0       0.66      0.66      0.66       393
         1.0       0.66      0.66      0.66       393

   micro avg       0.66      0.66      0.66       786
   macro avg       0.66      0.66      0.66       786
weighted avg       0.66      0.66      0.66       786



,tn,fp,fn,tp
0,260,133,132,261


### BaggingClassifier

In [176]:
from sklearn.ensemble import BaggingClassifier
modelo=BaggingClassifier(  n_estimators = 250 , max_samples = 1.0 ,
                           bootstrap = True , bootstrap_features = False ,
                         oob_score = False , warm_start = False , n_jobs = None , random_state = 0 , verbose = 0 )
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)


Correct classification rate: 0.7416020671834626
              precision    recall  f1-score   support

         0.0       0.76      0.71      0.73       387
         1.0       0.73      0.77      0.75       387

   micro avg       0.74      0.74      0.74       774
   macro avg       0.74      0.74      0.74       774
weighted avg       0.74      0.74      0.74       774



,tn,fp,fn,tp
0,276,111,89,298


### ExtraTreesClassifier

In [178]:
from sklearn.ensemble import ExtraTreesClassifier
modelo = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.7338501291989664
              precision    recall  f1-score   support

         0.0       0.75      0.70      0.72       387
         1.0       0.72      0.77      0.74       387

   micro avg       0.73      0.73      0.73       774
   macro avg       0.74      0.73      0.73       774
weighted avg       0.74      0.73      0.73       774



,tn,fp,fn,tp
0,269,118,88,299


## Redes neuronales

### MLPClassifier

In [179]:
from sklearn.neural_network import MLPClassifier
modelo=MLPClassifier(alpha=1)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.7558139534883721
              precision    recall  f1-score   support

         0.0       0.78      0.72      0.75       387
         1.0       0.74      0.79      0.76       387

   micro avg       0.76      0.76      0.76       774
   macro avg       0.76      0.76      0.76       774
weighted avg       0.76      0.76      0.76       774



,tn,fp,fn,tp
0,278,109,80,307


In [180]:
from sklearn import linear_model
modelo== linear_model.SGDClassifier(max_iter=1000, tol=1e-3)

modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.7583979328165374
              precision    recall  f1-score   support

         0.0       0.76      0.76      0.76       387
         1.0       0.76      0.75      0.76       387

   micro avg       0.76      0.76      0.76       774
   macro avg       0.76      0.76      0.76       774
weighted avg       0.76      0.76      0.76       774



,tn,fp,fn,tp
0,295,92,95,292


# Select kbest

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
selectKBest = SelectKBest(chi2, 84)
selectKBest.fit(train_features, train_labels)
best_train_features = selectKBest.transform(train_features)


idxs_selected = selectKBest.get_support(indices=True)
#Ahora tan solo de toda las caracteristicas que tenemos selecionamos los indices especificos

best_train_features = pd.DataFrame(train_features).iloc[:,idxs_selected]
#Selected columns.
print(best_train_features.columns) # selected columns
